In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AAC import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHUgrad2024!"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

# Create a list of options for the rescue types
rescue_types = [
    {'label': 'None', 'value': 'none'},
    {'label': 'Water Rescue', 'value': 'water_rescue'},
    {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain_rescue'},
    {'label': 'Disaster Rescue or Individual Tracking', 'value': 'disaster_rescue'},
]

#Header and branding logo

app.layout = html.Div([
    html.Div([
        html.A(
            href='https://www.snhu.edu',
            target='_blank',
            children=[
                html.Img(
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={
                        'maxWidth': '150px',
                        'maxHeight': '150px',
                        'float': 'left',
                        'marginRight': '10px'
                    }
                ),
                html.H1('Willem Kroeger - Project Two Dashboard', style={'display': 'inline-block'})
            ]
        )
    ]),
    html.Hr(),
    html.Div(),


#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
          # Interactive filter options
        html.Label('Rescue Type:'),
        dcc.RadioItems(
            id='rescue-type',
            options=rescue_types,
            value='None'
        ),
        html.Label('Breed:'),
        dcc.Dropdown(
            id='breed-dropdown',
            options=[],
            multi=True,
            placeholder='Select a breed'
        ),
    
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         
                         style_data={
            'whiteSpace': 'normal',
            'height': 'auto'
        },
        page_size=10,  # Set the number of rows displayed per page
        row_selectable='single',  # Enable single-row selection
        sort_action='native',  # Enable sorting
        sort_mode='multi',  # Enable multiple columns sorting

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
        html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



@app.callback(
    Output('datatable-id', 'data'),
    Output('datatable-id', 'columns'),
    [Input('rescue-type', 'value'),
     Input('breed-dropdown', 'value')]
)
def update_dashboard(rescue_type, breed):
    if rescue_type == 'water_rescue':
        filtered_df = df[
            (df['breed'].str.contains('Labrador Retriever|Chesa Bay Retr|Newfoundland', case=False, na=False)) &
            (df['sex_upon_outcome'] == 'Intact Female') &
            (df['age_upon_outcome_in_weeks'].between(26, 156))
        ]
        if breed:
            filtered_df = filtered_df[filtered_df['breed'].str.contains('|'.join(breed), case=False, na=False, regex=True)]
    elif rescue_type == 'mountain_rescue':
        filtered_df = df[
            (df['breed'].str.contains('German Shepherd|Alaskan Malamute|Old English Sheepdog|Siberian Husky|Rottweiler', case=False, na=False)) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'].between(26, 156))
        ]
        if breed:
            filtered_df = filtered_df[filtered_df['breed'].str.contains('|'.join(breed), case=False, na=False, regex=True)]
    elif rescue_type == 'disaster_rescue':
        filtered_df = df[
            (df['breed'].str.contains('Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler', case=False, na=False)) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'].between(20, 300))
        ]
        if breed:
            filtered_df = filtered_df[filtered_df['breed'].str.contains('|'.join(breed), case=False, na=False, regex=True)]
    else:
        filtered_df = df

    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in filtered_df.columns]
    data = filtered_df.to_dict('records')

    return data, columns


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return []

    filtered_df = pd.DataFrame(viewData)

    # Count the occurrences of each breed
    breed_counts = filtered_df['breed'].value_counts()

    # Create the pie chart figure
    fig = px.pie(breed_counts, values=breed_counts.values, names=breed_counts.index, title='Preferred Animals')

    # Update the layout of the pie chart
    fig.update_traces(textposition='inside', textinfo='percent+label')

    return [
        dcc.Graph(figure=fig)
    ]

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    # Convert viewData to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]

    # Create the geolocation chart with a map, tile layer, and marker
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                             children=[
                                 dl.Tooltip(dff.iloc[row, 4]),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(dff.iloc[row, 9])
                                         ])
                             ])
               ])
            ]

# Callback to update breed dropdown options
@app.callback(
    Output('breed-dropdown', 'options'),
    [Input('rescue-type', 'value')]
)
def update_breed_dropdown(rescue_type):
    breed_options = []
    
    if rescue_type == 'water_rescue':
        breed_options += [
            {'label': 'Labrador Retriever Mix', 'value': 'Labrador Retriever Mix'},
            {'label': 'Chesapeake Bay Retriever', 'value': 'Chesa Bay Retr'},
            {'label': 'Newfoundland', 'value': 'Newfoundland'}
        ]
    
    elif rescue_type == 'mountain_rescue':
        breed_options += [
            {'label': 'German Shepherd', 'value': 'German Shepherd'},
            {'label': 'Alaskan Malamute', 'value': 'Alaskan Malamute'},
            {'label': 'Old English Sheepdog', 'value': 'Old English Sheepdog'},
            {'label': 'Siberian Husky', 'value': 'Siberian Husky'},
            {'label': 'Rottweiler', 'value': 'Rottweiler'}
        ]
    
    elif rescue_type == 'disaster_rescue':
        breed_options += [
            {'label': 'Doberman Pinscher', 'value': 'Doberman Pinscher'},
            {'label': 'German Shepherd', 'value': 'German Shepherd'},
            {'label': 'Golden Retriever', 'value': 'Golden Retriever'},
            {'label': 'Bloodhound', 'value': 'Bloodhound'},
            {'label': 'Rottweiler', 'value': 'Rottweiler'}
        ]
    
    return breed_options




app.run_server(debug=True)


Connection Successful
Dash app running on http://127.0.0.1:18809/
